In [ ]:
# default_exp data.modules

In [ ]:
# hide
%load_ext autoreload
%autoreload 2

In [ ]:
# export
import pytorch_lightning as pl
from grade_classif.imports import *
from grade_classif.data.transforms import *
from grade_classif.data.utils import show_img, LabelSlideBalancedRandomSampler
from grade_classif.data.dataset import (
    ImageClassifDataset,
    MILDataset,
    NormDataset,
    RNNSlideDataset,
    FeaturesClassifDataset
)
from grade_classif.core import ifnone
from torch.utils.data import DataLoader, RandomSampler, WeightedRandomSampler

In [ ]:
# export
class BaseDataModule(pl.LightningDataModule):
    def __init__(
        self,
        datafolder: Path,
        data_csv: Path,
        batch_size: int = 32,
        size: int = 299,
        transforms: Optional[int] = None,
        patch_classes: Optional[Path] = None,
        concepts: Optional[Path] = None,
        concept_classes: Optional[Path] = None,
        train_percent: float = 1,
        get_id: Optional[Callable[[Any], str]] = None,
        pacs_filt: Optional[str] = None,
        num_workers: int = 4,
        filterfile: Optional[str] = None,
        val_fold: Optional[int] = None,
        **kwargs,
    ):
        super().__init__()
        self.datafolder = datafolder
        self.data_csv = data_csv
        self.batch_size = batch_size
        self.patch_classes = patch_classes
        self.concepts = concepts
        self.concept_classes = concept_classes
        self.size = size
        self.transforms = transforms
        self.train_percent = train_percent
        self.get_id = get_id
        self.pacs_filt = pacs_filt
        self.num_workers = num_workers
        self.filterfile = filterfile
        self.val_fold = val_fold
        if transforms is not None:
            if transforms < 10:
                tfm_func = globals()[f"get_transforms{transforms}"]
                self.tfms = tfm_func(size)
            else:
                tfm_func = globals()[f"get_transforms{10}"]
                self.tfms = tfm_func(size, n_inputs=transforms - 10)
        else:
            self.tfms = None

    def train_dataloader(self) -> DataLoader:
        return DataLoader(
            self.data.train,
            batch_size=self.batch_size,
            shuffle=True,
            drop_last=True,
            num_workers=self.num_workers,
            pin_memory=True,
        )

    def val_dataloader(self) -> DataLoader:
        # OPTIONAL
        # can also return a list of val dataloaders
        return DataLoader(
            self.data.valid, batch_size=self.batch_size, num_workers=4, pin_memory=True
        )

    def test_dataloader(self) -> DataLoader:
        # OPTIONAL
        # can also return a list of test dataloaders
        if self.data.test is not None:
            return DataLoader(
                self.data.test,
                batch_size=self.batch_size,
                num_workers=self.num_workers,
                pin_memory=True,
            )
        else:
            return None

    """def get_pacs_filt(self) -> Optional[Callable[[Path], bool]]:
        if self.patch_classes is not None:
            patch_classes_df = pd.read_csv(self.patch_classes, index_col="patchId")
            x_type = patch_classes_df.loc[x.stem, "type"]
            if self.filt != "all":

                def filt(x):
                    return x_type == self.filt

            else:

                def filt(x):
                    return x_type != "garb"

        elif self.concepts is not None and self.concept_classes is not None:
            conc_classes_df = pd.read_csv(self.concept_classes, index_col=0)
            if self.filt != "all":
                ok = conc_classes_df.loc[
                    conc_classes_df["type"] == self.filt
                ].index.values
            else:
                ok = conc_classes_df.loc[conc_classes_df["type"] != "garb"].index.values
            conc_df = pd.read_csv(self.concepts, index_col="patchId")

            def filt(x):
                return conc_df.loc[x.stem, "concept"] in ok

        else:
            filt = None
        return filt"""
    
    def get_pacs_filt(self) -> Optional[Callable[[Path], bool]]:
        if self.pacs_filt is None:
            return None
        else:
            def filt(x):
                for pacs in self.pacs_filt:
                    if f"PACS0{pacs}" in x.name:
                        return True
                return False
            return filt
        
    def get_accepted_files(self) -> Union[None, Set[str]]:
        if self.filterfile is None:
            return None
        else:
            with open(self.filterfile, "r") as f:
                accepted_files = f.read().split("\n")
            return set(accepted_files)

    def show_some(self, n: int = 8, split: str = "train", imgsize: int = 4):
        fig, axs = plt.subplots(n, 2, figsize=(imgsize * 2, imgsize * n))
        data = getattr(self.data, split)
        idxs = np.random.choice(np.arange(len(data)), size=n, replace=False)
        for ax_r, idx in zip(axs, idxs):
            x, x_tfmed = data.get_orig_tfmed(idx)
            show_img(x, ax=ax_r[0])
            show_img(x_tfmed, ax=ax_r[1])
        title = "original/transformed"
        fig.suptitle(title)
        plt.show()

In [ ]:
# export
class NormDataModule(BaseDataModule):
    def setup(self, stage: Optional[str] = None):
        self.data = (
            NormDataset.from_folder(
                self.datafolder,
                extensions=[".png"],
                train_percent=self.train_percent,
                div=False,
            )
            .split_by_csv(self.data_csv, get_id=self.get_id, val_fold=self.val_fold)
            .to_tensor(tfms=self.tfms)
        )

In [ ]:
# export
class ImageClassifDataModule(BaseDataModule):
    def __init__(
        self,
        datafolder: Path,
        data_csv: Path,
        classes: Sequence[str],
        label_func: Callable[[Path], str],
        sample_mode: int = 0,
        max_patches_per_slide: Optional[int] = None,
        norm_ref : Optional[str] = None,
        norm_method : Optional[str] = None,
        include : Optional[Sequence[str]] = None,
        **kwargs
    ):
        super().__init__(datafolder, data_csv, **kwargs)
        self.classes = classes
        self.n_classes = len(classes)
        self.sample_mode = sample_mode
        self.label_func = label_func
        self.max_patches_per_slide = max_patches_per_slide
        self.norm_ref = norm_ref
        self.norm_method = norm_method
        self.include=include

    def setup(self, stage: Optional[str] = None):
        if not hasattr(self, "data"):
            self.filt = self.get_pacs_filt()
            data = ImageClassifDataset.from_folder(
                self.datafolder,
                self.label_func,
                classes=self.classes,
                extensions=[".png"],
                include=self.include,
                filterfunc=self.filt,
                accepted_files=self.get_accepted_files(),
                train_percent=self.train_percent,
                norm_ref = self.norm_ref,
                norm_method = self.norm_method,
                div=False
            ).split_by_csv(self.data_csv, get_id=self.get_id, val_fold=self.val_fold)
            self.data = data.to_tensor(tfms=self.tfms, tfm_y=False)

    def get_slide_weights(self) -> NDArray[(Any,), float]:
        items = self.data.train.items
        get_slide = np.vectorize(lambda x: "_".join(x.name.split("_")[:-2]))
        all_slides = get_slide(items)
        slides = np.unique(all_slides)
        n = len(items)
        weights = np.ones_like(items, dtype=np.float32)
        n_patches_max = 0
        for slide in slides:
            slide_mask = all_slides == slide
            n_patches = slide_mask.sum()
            weight = n / n_patches
            weights[slide_mask] = weight
            if self.sample_mode == 1:
                n_patches_max += self.max_patches_per_slide
            else:
                n_patches_max += min(self.max_patches_per_slide, n_patches)
        return weights, n_patches_max

    def get_n_patches_max(self, all_slides: NDArray[(Any,), str]) -> int:
        slides = np.unique(all_slides)
        if self.sample_mode == 1:
            return len(slides) * self.max_patches_per_slide
        else:
            n_patches_max = 0
            for slide in slides:
                n_patches = (all_slides == slide).sum()
                n_patches_max += min(n_patches, self.max_patches_per_slide)
            return n_patches_max

    def train_dataloader(self) -> DataLoader:
        sm = self.sample_mode
        labels = self.data.train.labels
        items = self.data.train.items
        get_slide = np.vectorize(lambda x: "_".join(x.name.split("_")[:-2]))
        all_slides = get_slide(items)
        if sm > 0:
            if self.max_patches_per_slide is not None:
                n_patches_max = self.get_n_patches_max(all_slides)
            else:
                n_patches_max = len(labels)
            if sm == 1:
                n = max([(labels == cl).sum() for cl in self.classes])
                replacement = True
            else:
                n = min([(labels == cl).sum() for cl in self.classes])
                replacement = False
            n_patches = int(self.n_classes * n * n_patches_max / len(labels))
            sampler = LabelSlideBalancedRandomSampler(
                labels, all_slides, n_patches, replacement=replacement
            )
        else:
            sampler = RandomSampler(self.data.train)
        return DataLoader(
            self.data.train,
            batch_size=self.batch_size,
            sampler=sampler,
            drop_last=True,
            num_workers=self.num_workers,
            pin_memory=True,
        )

In [ ]:
# export
class FeaturesClassifDataModule(BaseDataModule):
    def __init__(
        self,
        datafolder: Path,
        data_csv: Path,
        classes: Sequence[str],
        label_func: Callable[[Path], str],
        sample_mode: int = 0,
        embed_dim: int = 2048,
        padding_file: Optional[Path] = None,
        **kwargs
    ):
        super().__init__(datafolder, data_csv, **kwargs)
        self.classes = classes
        self.n_classes = len(classes)
        self.sample_mode = sample_mode
        self.label_func = label_func
        self.embed_dim = embed_dim
        self.padding_file = padding_file

    def setup(self, stage: Optional[str] = None):
        if not hasattr(self, "data"):
            self.filt = self.get_pacs_filt()
            data = FeaturesClassifDataset.from_folder(
                self.datafolder,
                self.label_func,
                classes=self.classes,
                extensions=[".csv"],
                include=self.classes,
                filterfunc=self.filt,
                train_percent=self.train_percent,
                size=self.size,
                embed_dim=self.embed_dim,
                padding_file = self.padding_file
            ).split_by_csv(self.data_csv, get_id=self.get_id, val_fold=self.val_fold)
            self.data = data.to_tensor()

    def train_dataloader(self) -> DataLoader:
        sm = self.sample_mode
        labels = self.data.train.labels
        if sm > 0:
            weights = torch.zeros(len(labels), dtype=torch.float32)
            for k, cl in enumerate(self.classes):
                mask = labels == cl
                weights[mask] = len(labels) / mask.sum()
            if sm == 1:
                n = max([(labels == cl).sum() for cl in self.classes])
            else:
                n = min([(labels == cl).sum() for cl in self.classes])
            n_patches = int(self.n_classes * n)
            sampler = WeightedRandomSampler(weights, n_patches, replacement=True)
        else:
            sampler = RandomSampler(self.data.train)
        return DataLoader(
            self.data.train,
            batch_size=self.batch_size,
            sampler=sampler,
            drop_last=True,
            num_workers=self.num_workers,
            pin_memory=True,
        )

In [ ]:
# export
class MILDataModule(BaseDataModule):
    def __init__(
        self,
        datafolder: Union[str, Path],
        data_csv: Union[str, Path],
        coord_csv: Union[str, Path],
        classes: Sequence[str],
        label_func: Optional[Callable[[Path], str]] = None,
        extensions: Optional[Sequence[str]] = None,
        level: int = 1,
        sample_mode: int = 0,
        **kwargs
    ):
        super().__init__(datafolder, data_csv, **kwargs)
        assert len(classes) == 2, "MIL only works with 2 classes."
        self.sample_mode = sample_mode
        self.coord_csv = coord_csv
        self.classes = classes
        self.label_func = ifnone(label_func, lambda x: x.parent.name)
        self.extensions = ifnone(extensions, [".mrxs"])
        self.level = level

    def setup(self, stage: Optional[str] = None):
        self.data = (
            MILDataset.from_folder(
                self.datafolder,
                self.label_func,
                self.coord_csv,
                classes=self.classes,
                extensions=self.extensions,
                include=self.classes,
                train_percent=self.train_percent,
                size=self.size,
                level=self.level,
            )
            .split_by_csv(self.data_csv)
            .to_tensor(tfms=self.tfms, tfm_y=False)
        )

    def train_dataloader(self) -> DataLoader:
        sm = self.sample_mode
        w = np.float32(
            (self.data.train.labels == self.classes[0]).sum()
            / (self.data.train.labels == self.classes[1]).sum()
        )
        min_class = 1
        if w < 1:
            w = 1 / w
            min_class = 0
        if sm > 0:
            labels = self.data.train.labels == self.classes[min_class]
            weights = np.where(labels, w, 1.0)
            if sm == 1:
                sampler = WeightedRandomSampler(weights, 2 * len(np.argwhere(~labels)))
            else:
                sampler = WeightedRandomSampler(
                    weights, 2 * len(np.argwhere(labels))
                )
        else:
            sampler = RandomSampler(self.data.train)
        return DataLoader(
            self.data.train,
            batch_size=self.batch_size,
            sampler=sampler,
            drop_last=True,
            pin_memory=True,
        )

    def val_dataloader(self) -> DataLoader:
        # OPTIONAL
        # can also return a list of val dataloaders
        return DataLoader(
            self.data.valid,
            batch_size=self.batch_size,
            pin_memory=True,
        )

    def test_dataloader(self) -> DataLoader:
        return DataLoader(
            self.data.test,
            batch_size=self.batch_size,
            shuffle=False,
            pin_memory=True,
        )

In [ ]:
# export
class RNNAggDataModule(MILDataModule):
    def __init__(
        self,
        datafolder: Union[str, Path],
        data_csv: Union[str, Path],
        coord_csv: Union[str, Path],
        classes: Sequence[str],
        patches_per_slide=10,
        **kwargs
    ):
        super().__init__(datafolder, data_csv, coord_csv, classes, **kwargs)
        self.patches_per_slide = patches_per_slide

    def setup(self, stage: Optional[str] = None):
        self.data = (
            RNNSlideDataset.from_folder(
                self.datafolder,
                self.label_func,
                self.coord_csv,
                classes=self.classes,
                extensions=self.extensions,
                include=self.classes,
                train_percent=self.train_percent,
                size=self.size,
                level=self.level,
                patches_per_slide=self.patches_per_slide,
            )
            .split_by_csv(self.data_csv)
            .to_tensor(tfms=self.tfms, tfm_y=False)
        )
        weights = np.float32(
            (self.data.train.labels == self.classes[0]).sum()
            / (self.data.train.labels == self.classes[1]).sum()
        )
        min_class = 1
        if weights < 1:
            weights = 1 / weights
            min_class = 0
        self.min_class = min_class
        self.weights = weights

In [ ]:
# hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_train.ipynb.
Converted 02_predict.ipynb.
Converted 10_data.read.ipynb.
Converted 11_data.loaders.ipynb.
Converted 12_data.dataset.ipynb.
Converted 13_data.utils.ipynb.
Converted 14_data.transforms.ipynb.
Converted 15_data.color.ipynb.
Converted 16_data.modules.ipynb.
Converted 20_models.plmodules.ipynb.
Converted 21_models.modules.ipynb.
Converted 22_models.utils.ipynb.
Converted 23_models.hooks.ipynb.
Converted 24_models.metrics.ipynb.
Converted 25_models.losses.ipynb.
Converted 80_params.defaults.ipynb.
Converted 81_params.parser.ipynb.
Converted 99_index.ipynb.
